In [1]:
!pip install git+https://github.com/openai/whisper.git -q


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00


In [3]:
!pip install -q langchain accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00


In [4]:
import whisper
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [5]:
model=whisper.load_model("medium")

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 103MiB/s]


In [6]:
def add_text(audio):
    #time.sleep(3)
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    options = {"fp16": False, "language": "hi", "task": "translate"}
    result = whisper.decode(model, mel, **options)
    text = result.text
    #state += text + " "
    return text

In [7]:
#text=add_text("/content/Wednesday at 12-02 PM.m4a")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)b-chat-hf/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
llm_model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              load_in_4bit=True,
                                              bnb_4bit_quant_type="nf4",
                                              bnb_4bit_compute_dtype=torch.float16)


(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [13]:
!pip install ctransformers[cuda]

In [14]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=llm_model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [28]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced clinical bot and medical expert.
you will use your domain knowledge in medical science to retrieve information  for a certain section only if the information regading a particular section is present in the conversation of patient and doctor in medical note of a certain format or provide 'NA' if no relevant information regading a certain section is present in the conversation of patient and doctor after carefully analysing the conversation between patient and doctor
kindly respond 'the conversation is not related to medical domain' if no conversation between patient and doctor is not related to any medical domain and do not generate any clinical note for these type of conversation
"""

In [29]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [30]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [18]:
"""
Based on the conversation transcript and doctor's hints provided below, generate a clinical note in the following format:
Diagnosis:
History of Presenting Illness:
Medications (Prescribed): List current medications and note if they are being continued, or if any new ones have been added.
Lab Tests (Ordered):
Please consider any information in the transcript that might be relevant to each of these sections, and use the doctor's hint as a guide.

### Example
Conversation Transcript:
Patient: “I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.”
Doctor: “I see, we may need to adjust your medications. Let's add Jalra-OD and Telmis to your regimen and see how you respond.”
Doctor's Hint: The patient has uncontrolled diabetes and hypertension despite adherence to the Glycomet-GP 1.
Clinical Note:
Diagnosis: Uncontrolled Diabetes and Hypertension
History of Presenting Illness: The patient has been adhering to their current medication regimen but the diabetes and hypertension seem uncontrolled.
Medications (Prescribed):
[Continue] Glycomet-GP 1 (tablet) | Glimepiride and Metformin
[Added] Jalra-OD 100mg (tablet) | Vildagliptin
[Added] Telmis 20 (Tablet)
Lab Tests (Ordered): None
Now, based on the following conversation and hints, please generate a clinical note:

### Conversation Transcript
{transcript}

"""

"\nBased on the conversation transcript and doctor's hints provided below, generate a clinical note in the following format:\nDiagnosis:\nHistory of Presenting Illness:\nMedications (Prescribed): List current medications and note if they are being continued, or if any new ones have been added.\nLab Tests (Ordered):\nPlease consider any information in the transcript that might be relevant to each of these sections, and use the doctor's hint as a guide.\n\n### Example\nConversation Transcript:\nPatient: “I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.”\nDoctor: “I see, we may need to adjust your medications. Let's add Jalra-OD and Telmis to your regimen and see how you respond.”\nDoctor's Hint: The patient has uncontrolled diabetes and hypertension despite adherence to the Glycomet-GP 1.\nClinical Note:\nDiagnosis: Uncontrolled Diabetes and Hypertension\nHisto

In [144]:
system_prompt = """
You are an advanced clinical bot and medical expert.
you will use your domain knowledge in medical science to retrieve information  for a certain section only if the information regading a particular section is present in the conversation of patient and doctor in medical note of a certain format or provide 'NA' if no relevant information regading a certain section is present in the conversation of patient and doctor after carefully analysing the conversation between patient and doctor
kindly respond 'the conversation is not related to medical domain' if no conversation between patient and doctor is not related to any medical domain and do not generate any clinical note for these type of conversation
"""

instruction = """
Conversation Between Patient and Doctors-
Patient: “I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.”
Doctor: “I see, we may need to adjust your medications. Let's add Jalra-OD and Telmis to your regimen and see how you respond.”

use your domain knowledge in medical science to retrieve information  for a certain section only if the information regading  particular sections is present in the conversation of patient and doctor in medical note of a format mentioned below  else provide 'NA' if no relevant information regading a certain section is present in the conversation of patient and doctor after carefully analysing the conversation between patient and doctor
kindly respond 'the conversation is not related to medical domain' if conversation between patient and doctor is not related to any medical domain and do not generate any clinical note for these type of conversation

the format of clinical note is :

Diagnosis:

History of Presenting Illness:

Medications (Prescribed):

Lab Tests (Ordered):


### Example only for reference puposes


Patient: I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.

Doctor: I see, we may need to adjust your medications. Let's add Jalra-OD and Telmis to your regimen and see how you respond.

###

Clinical Note :-

Diagnosis: Uncontrolled Diabetes and Hypertension

History of Presenting Illness: The patient has been adhering to their current medication regimen but the diabetes and hypertension seem uncontrolled.

Medications (Prescribed):

Glycomet-GP 1

Jalra-OD 100mg

Telmis 20

Lab Tests (Ordered): None

###

Now, based on the following conversation pleas below please generate a clinical note:

Conversation Between Patient and Doctors :-\n\n{text}

"""
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>

You are an advanced clinical bot and medical expert.
you will use your domain knowledge in medical science to retrieve information  for a certain section only if the information regading a particular section is present in the conversation of patient and doctor in medical note of a certain format or provide 'NA' if no relevant information regading a certain section is present in the conversation of patient and doctor after carefully analysing the conversation between patient and doctor 
kindly respond 'the conversation is not related to medical domain' if no conversation between patient and doctor is not related to any medical domain and do not generate any clinical note for these type of conversation

<>


Conversation Between Patient and Doctors-
Patient: “I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.”
Doctor: “I see, we may need to adjust your 

In [145]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [146]:
response = llm_chain.run("What's your interest in it Mr. Wayne? I want to borrow it for spelunking. Spelunking? Yeah, you know, cave diving. You're expecting to run into much gunfire in these caves?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  I apologize, but the conversation you provided does not relate to any medical domain. The conversation is about borrowing a car for spelunking (cave diving) and does not involve any medical conditions or treatment. Therefore, I cannot generate a clinical note for this conversation.

Please provide a conversation that is related to medical issues, and I will be happy to help you with a clinical note.


In [150]:
response = llm_chain.run("Patient's name is Adhiraj Singh Shikhaward. Patient's symptoms are dizziness, cough and cold. I am prescribing medicines to the patient. Paracetamol, Citrazin and Azithromycin.")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Clinical Note:

Diagnosis: Acute Sinusitis and Common Cold

History of Presenting Illness: The patient, Adhiraj Singh Shikhaward, is experiencing dizziness, cough, and cold symptoms.

Medications (Prescribed):

Paracetamol 500mg 2-3 times a day

Citrazin 500mg 3 times a day

Azithromycin 500mg 1 time a day

Lab Tests (Ordered): None

NA - There is no relevance to medical domain in this conversation, therefore no clinical note will be generated.


In [149]:
response = llm_chain.run(
"""
Patient: I am having dizziness, cough and cold. what should i do?
Doctor: take Paracetamol, Citrazin and Azithromycin.
""")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  Based on the provided conversation between the patient and doctor, I can generate a clinical note for the following sections:

Diagnosis:

History of Presenting Illness:

Medications (Prescribed):

Lab Tests (Ordered):

Clinical Note:

Diagnosis: Acute Sinusitis, Cold and Cough

History of Presenting Illness: The patient has developed new symptoms of dizziness, cough, and cold.

Medications (Prescribed): Paracetamol 500mg TDS, Citrazin 500mg TDS, Azithromycin 500mg BD

Lab Tests (Ordered): None

Note: The conversation is related to medical domain as the patient is seeking medical advice for their symptoms, and the doctor is prescribing medications to treat the condition. Therefore, I have provided a clinical note for the above sections.


In [36]:
import time
import random

In [129]:
def add_text(audio):
    time.sleep(3)
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)
    options = {"fp16": False, "language": "hi", "task": "translate"}
    result = whisper.decode(model, mel, **options)
    text = result.text
    #state += text + " "
    return text

In [130]:
add_text("/content/Wednesday at 12-02 PM.m4a")

"Patient's name is Adhiraj Singh Shikhaward. Patient's symptoms are dizziness, cough and cold. I am prescribing medicines to the patient. Paracetamol, Citrazin and Azithromycin."

In [131]:
def generate_response(query, chat_history: list) -> tuple:
    #messages = generate_messages(chat_history, query)
    bot_message = llm_chain.run(query)
    chat_history.append((query, bot_message))
    time.sleep(random.randint(0, 5))
    return '', chat_history

In [132]:
def user(user_message, history):
        return "", history + [[user_message, None]]



In [133]:
def bot(query,history):
        bot_message=llm_chain.run(query)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history


In [93]:
!pip install -q typing-extensions==4.5.0 ftfy bitsandbytes==0.35.0 gradio natsort

In [134]:
import gradio as gr

In [135]:
block=gr.Blocks()
with block:
    gr.Markdown("""<h1><center>Chikitsa Bot</center></h1>
    """)
    chatbot = gr.Chatbot()
    with gr.Row():
        message=gr.Textbox()
        with gr.Column():
             btn = gr.components.Audio(source="microphone", type="filepath")
             upload = gr.Audio(sources=["upload"], type="filepath")
             submit = gr.Button("SUBMIT")
    submit.click(add_text, inputs=[btn], outputs=[message]),
    submit.click(add_text,inputs=[upload],outputs=[message])
    message.submit(generate_response, [message, chatbot], [message, chatbot]),


block.queue()
block.launch()

<ipython-input-135-25aca7d65c3b>:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': ['upload']}
  upload = gr.Audio(sources=["upload"], type="filepath")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8ccfb6eb3b49a508b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [136]:
block=gr.Blocks()
with block:
    gr.Markdown("""<h1><center>Chikitsa Bot</center></h1>
    """)
    chatbot = gr.Chatbot()
    with gr.Row():
        message=gr.Textbox()
        with gr.Column():
            btn = gr.components.Audio(source="microphone", type="filepath")
            upload = gr.Audio(sources=["upload"], type="filepath")
            submit = gr.Button("SUBMIT")
    submit.click(add_text, inputs=[btn], outputs=[message]),
    submit.click(add_text,inputs=[upload],outputs=[message])
    message.submit(user, [message, chatbot], [message, chatbot], queue=False).then(
        bot, [message,chatbot],chatbot
    )
block.queue()
block.launch()

<ipython-input-136-3a11c647a617>:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': ['upload']}
  upload = gr.Audio(sources=["upload"], type="filepath")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c72cc6a8f881ac58ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
